In [1]:
from IPython import display

from utils import Logger

import torch
from torch import nn
from torch.optim import Adam

from torchvision import transforms, datasets

In [2]:
DATA_FOLDER = './torch_data/CIFAR'